# Import modules

In [2]:
import pandas as pd
import numpy as np


# Loading dataset

In [3]:
data=pd.read_csv('C:\\Users\\USER\\Desktop\\elast\\GOOGL.csv')
data.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,15.689439,15.753504,15.621622,15.684434,15.684434,78169752
1,2010-01-05,15.695195,15.711712,15.554054,15.615365,15.615365,120067812
2,2010-01-06,15.662162,15.662162,15.174174,15.221722,15.221722,158988852
3,2010-01-07,15.250250,15.265265,14.831081,14.867367,14.867367,256315428
4,2010-01-08,14.814815,15.096346,14.742492,15.065566,15.065566,188783028


In [4]:
data.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [5]:
data.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [7]:
data['Date']=pd.to_numeric(data['Date'],errors='coerce')

In [9]:
#calculate the simple returns
daily_simple_returns=data.pct_change(1)
daily_simple_returns

,Date,Open,High,Low,Close,Adj Close,Volume
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.000367,-0.002653,-0.004325,-0.004404,-0.004404,0.535988
2,NaN,-0.002105,-0.003154,-0.024423,-0.025209,-0.025209,0.324159
3,NaN,-0.026300,-0.025341,-0.022610,-0.023280,-0.023280,0.612160
4,NaN,-0.028553,-0.011066,-0.005973,0.013331,0.013331,-0.263474
...,...,...,...,...,...,...,...
3266,NaN,-0.010328,-0.018621,-0.022645,-0.020317,-0.020317,0.117706
3267,NaN,-0.011910,0.011407,0.008689,0.016750,0.016750,-0.168315
3268,NaN,0.019401,-0.006812,-0.000689,-0.020621,-0.020621,-0.126318
3269,NaN,-0.020495,-0.010119,-0.012297,-0.015677,-0.015677,-0.028566


In [10]:
#calculate the mean  and show of simple daily returns
dailyMeanSimpleReturns=daily_simple_returns.mean()
dailyMeanSimpleReturns

Date              NaN
Open         0.000663
High         0.000637
Low          0.000646
Close        0.000674
Adj Close    0.000674
Volume       0.061879
dtype: float64

In [11]:
#show the annual returns
dailyMeanSimpleReturns*252

Date               NaN
Open          0.167015
High          0.160462
Low           0.162849
Close         0.169728
Adj Close     0.169728
Volume       15.593569
dtype: float64

In [16]:
#sort values for daily mean returns and show the annualized returns as percentage

dailyMeanSimpleReturns.sort_values(axis=0,ascending=False,inplace=True,kind='quicksort',na_position='last',ignore_index='False',key=None)

dailyMeanSimpleReturns*252*100

0    1559.356932
1      16.972812
2      16.972812
3      16.701459
4      16.284861
5      16.046211
6            NaN
dtype: float64

In [17]:
pip install pyportfolioOpt

In [18]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [20]:
#optimize the portfolio
#calculate the expected returns and the annulized sample covarience matrix of(daily) assets returns
mu=expected_returns.mean_historical_return(data)
S=risk_models.sample_cov(data)

In [23]:
#optimize for maximal Sharpe ratio
ef=EfficientFrontier(mu,S)
weights=ef.max_sharpe()
cleaned_weights=ef.clean_weights()
ef.portfolio_performance(verbose=True)
print(cleaned_weights)

Expected annual return: 14.2%
Annual volatility: 22.2%
Sharpe Ratio: 0.55
OrderedDict([('Open', 0.2923), ('High', 0.3365), ('Low', 0.07422), ('Close', 0.14849), ('Adj Close', 0.14849), ('Volume', 0.0)])


In [24]:
pip install pulp

In [26]:
#Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation
portfolio_val=1000
latest_prices=data.tail(1).squeeze()
weights=cleaned_weights
da=DiscreteAllocation(weights,latest_prices,total_portfolio_value=portfolio_val)
allocation,leftover=da.lp_portfolio()
print('Discrete allocation:',allocation)
print('Funds remaining:$',leftover)

Discrete allocation: {'High': 4, 'Low': 1, 'Close': 2, 'Adj Close': 2}
Funds remaining:$ 204.19001898526324


In [27]:
# Get the discrete allocation values
discrete_allocation_list=[]
for symbol in allocation:
    discrete_allocation_list.append(allocation.get(symbol))

In [29]:
#Create  a new DataFrame for portfolio
portfolio_df=pd.DataFrame(columns=['company_ticker','Discrete_val'+str(portfolio_val)])

#Add data to dataframe
portfolio_df['company_ticker']=allocation
portfolio_df['Discrete_val'+str(portfolio_val)]=discrete_allocation_list
portfolio_df['Price']=latest_prices[allocation].values

C:\Users\USER\AppData\Local\Temp\ipykernel_7928\3829392893.py:7: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  portfolio_df['Price']=latest_prices[allocation].values


In [31]:
#show the portfolio
portfolio_df

,company_ticker,Discrete_val1000,Price
High,High,4,88.849998
Low,Low,1,86.610001
Close,Close,2,88.449997
Adj Close,Adj Close,2,88.449997
